# CNN Classification with MPQA Dataset
<hr>

The __modus operandi__ for text classification is to use __word embedding__ for representing words and a Convolutional neural network to learn how to discriminate documents on classification problems. 

__Yoav Goldberg__ commented in _A Primer on Neural Network Models for Natural Language Processing, 2015._ :
> _The non-linearity of the network, as well as the ability to easily integrate pre-trained
word embeddings, often lead to superior classification accuracy._

He also commented in _Neural Network Methods for Natural Language Processing, 2017_ :
> ... _the CNN is in essence a feature-extracting architecture. ... . The CNNs layer's responsibility is to extract meaningful sub-structures that are useful for the overall prediction task at hand._

We will build a text classification model using CNN model on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 1,2, 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2) constraint: 3.
- Batch Size: 50.
- Update Rule: Adam

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/SUBJ/SUBJ.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10000, 3)


,sentence,label,split
0,"smart and alert , thirteen conversations about...",0,train
1,"color , musical bounce and warm seas lapping o...",0,train
2,it is not a mass market entertainment but an u...,0,train
3,a light hearted french film about the spiritua...,0,train
4,my wife is an actress has its moments in looki...,0,train
...,...,...,...
9995,"in the end , they discover that balance in lif...",1,train
9996,a counterfeit 1000 tomin bank note is passed i...,1,train
9997,enter the beautiful and mysterious secret agen...,1,train
9998,after listening to a missionary from china spe...,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10000 non-null  object
 1   label     10000 non-null  int32 
 2   split     10000 non-null  object
dtypes: int32(1), object(2)
memory usage: 195.4+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,5000,5000
1,5000,5000


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'smart and alert , thirteen conversations about one thing is a small gem .'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

print("Example of sentence: ", sentences[4])

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  my wife is an actress has its moments in looking at the comic effects of jealousy . in the end , though , it is only mildly amusing when it could have been so much more .
Into a sequence of int: [336, 208, 8, 16, 921, 25, 29, 312, 7, 313, 32, 2, 488, 551, 5, 3203, 7, 2, 129, 194, 10, 8, 60, 2330, 716, 39, 10, 128, 43, 82, 54, 81, 45]
Into a padded sequence: [ 336  208    8   16  921   25   29  312    7  313   32    2  488  551
    5 3203    7    2  129  194   10    8   60 2330  716   39   10  128
   43   82   54   81   45    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
a 3
and 4
of 5
to 6
in 7
is 8
's 9
it 10
21324


# Model 1: Embedding Random
<hr>

A __standard model__ for document classification is to use (quoted from __Jason Brownlee__, the author of [machinelearningmastery.com](https://machinelearningmastery.com)):
>- Word Embedding: A distributed representation of words where different words that have a similar meaning (based on their usage) also have a similar representation.
>- Convolutional Model: A feature extraction model that learns to extract salient features from documents represented using a word embedding.
>- Fully Connected Model: The interpretation of extracted features in terms of a predictive output.


Therefore, the model is comprised of the following elements:
- __Input layer__ that defines the length of input sequences.
- __Embedding layer__ set to the size of the vocabulary and 100-dimensional real-valued representations.
- __Conv1D layer__ with 32 filters and a kernel size set to the number of words to read at once.
- __MaxPooling1D layer__ to consolidate the output from the convolutional layer.
- __Flatten layer__ to reduce the three-dimensional output to two dimensional for concatenation.

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2): 3.
- Batch Size: 50.
- Update Rule: Adam

We will perform the best parameter using __grid search__ and 10-fold cross validation.

## CNN Model

Now, we will build Convolutional Neural Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from `CNN for Sentence Classification` by *Yoon Kim*.

Now, we will define our CNN model as follows:
- One Conv layer with 100 filters, kernel size 5, and relu activation function;
- One MaxPool layer with pool size = 2;
- One Dropout layer after flattened;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [11]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(filters = 100, kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [12]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          6397200   
_________________________________________________________________
conv1d (Conv1D)              (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 100)           0         
_________________________________________________________________
flatten (Flatten)            (None, 4900)              0         
_________________________________________________________________
dropout (Dropout)            (None, 4900)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                49010     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0

In [13]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [14]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1

            # Define the input shape
            model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=15, verbose=2, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

Epoch 1/15
180/180 - 19s - loss: 0.6851 - accuracy: 0.5187 - val_loss: 0.5145 - val_accuracy: 0.8610
Epoch 2/15
180/180 - 18s - loss: 0.3089 - accuracy: 0.8681 - val_loss: 0.2200 - val_accuracy: 0.9140
Epoch 3/15
180/180 - 17s - loss: 0.1218 - accuracy: 0.9559 - val_loss: 0.2579 - val_accuracy: 0.9090
Epoch 4/15
180/180 - 17s - loss: 0.0689 - accuracy: 0.9776 - val_loss: 0.3511 - val_accuracy: 0.9110
Epoch 5/15
180/180 - 18s - loss: 0.0463 - accuracy: 0.9864 - val_loss: 0.4157 - val_accuracy: 0.9040
Epoch 6/15
180/180 - 17s - loss: 0.0361 - accuracy: 0.9904 - val_loss: 0.4922 - val_accuracy: 0.9000
Epoch 7/15
180/180 - 17s - loss: 0.0322 - accuracy: 0.9892 - val_loss: 0.6074 - val_accuracy: 0.9090
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 91.39999747276306
Epoch 1/15
180/180 - 19s - loss: 0.5219 - accuracy: 0.7141 - val_loss: 0.2361 - val_accuracy: 0.9030
Epoch 2/15
180/180 - 18s - loss: 0.2007 - accuracy: 0.9270 - val_loss: 0.20

Epoch 5/15
180/180 - 18s - loss: 0.0494 - accuracy: 0.9768 - val_loss: 0.5339 - val_accuracy: 0.8960
Epoch 6/15
180/180 - 18s - loss: 0.0427 - accuracy: 0.9814 - val_loss: 0.5424 - val_accuracy: 0.8960
Epoch 7/15
180/180 - 18s - loss: 0.0345 - accuracy: 0.9829 - val_loss: 0.6695 - val_accuracy: 0.9030
Epoch 8/15
180/180 - 18s - loss: 0.0350 - accuracy: 0.9840 - val_loss: 0.6754 - val_accuracy: 0.8950
Epoch 9/15
180/180 - 18s - loss: 0.0327 - accuracy: 0.9836 - val_loss: 0.6963 - val_accuracy: 0.8960
Epoch 10/15
180/180 - 18s - loss: 0.0343 - accuracy: 0.9828 - val_loss: 0.7160 - val_accuracy: 0.8970
Epoch 11/15
180/180 - 18s - loss: 0.0319 - accuracy: 0.9847 - val_loss: 0.7615 - val_accuracy: 0.8920
Epoch 12/15
180/180 - 18s - loss: 0.0308 - accuracy: 0.9844 - val_loss: 0.8657 - val_accuracy: 0.8970
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 90.2999997138977

  Activation Filters       acc1  acc2  acc3  acc4       acc5       acc6 

Test Accuracy: 92.69999861717224
Epoch 1/15
180/180 - 20s - loss: 0.5901 - accuracy: 0.6478 - val_loss: 0.3184 - val_accuracy: 0.8930
Epoch 2/15
180/180 - 19s - loss: 0.3255 - accuracy: 0.9028 - val_loss: 0.2560 - val_accuracy: 0.9210
Epoch 3/15
180/180 - 19s - loss: 0.1935 - accuracy: 0.9436 - val_loss: 0.2204 - val_accuracy: 0.9230
Epoch 4/15
180/180 - 19s - loss: 0.1227 - accuracy: 0.9606 - val_loss: 0.2648 - val_accuracy: 0.9150
Epoch 5/15
180/180 - 18s - loss: 0.1036 - accuracy: 0.9644 - val_loss: 0.3502 - val_accuracy: 0.9130
Epoch 6/15
180/180 - 19s - loss: 0.0958 - accuracy: 0.9670 - val_loss: 0.3776 - val_accuracy: 0.9040
Epoch 7/15
180/180 - 18s - loss: 0.0964 - accuracy: 0.9633 - val_loss: 0.3954 - val_accuracy: 0.9140
Epoch 8/15
180/180 - 18s - loss: 0.0934 - accuracy: 0.9644 - val_loss: 0.3929 - val_accuracy: 0.9160
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 92.29999780654907
Epoch 1/15
180/180 - 20s - loss: 0.5481 - 

Epoch 5/15
180/180 - 20s - loss: 0.0334 - accuracy: 0.9816 - val_loss: 0.4313 - val_accuracy: 0.9130
Epoch 6/15
180/180 - 20s - loss: 0.0295 - accuracy: 0.9832 - val_loss: 0.4829 - val_accuracy: 0.9130
Epoch 7/15
180/180 - 20s - loss: 0.0285 - accuracy: 0.9812 - val_loss: 0.5717 - val_accuracy: 0.9060
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 92.59999990463257
Epoch 1/15
180/180 - 21s - loss: 0.5733 - accuracy: 0.6467 - val_loss: 0.2467 - val_accuracy: 0.9000
Epoch 2/15
180/180 - 20s - loss: 0.2482 - accuracy: 0.9192 - val_loss: 0.1988 - val_accuracy: 0.9230
Epoch 3/15
180/180 - 20s - loss: 0.1184 - accuracy: 0.9672 - val_loss: 0.2618 - val_accuracy: 0.9100
Epoch 4/15
180/180 - 20s - loss: 0.0715 - accuracy: 0.9804 - val_loss: 0.3669 - val_accuracy: 0.9120
Epoch 5/15
180/180 - 20s - loss: 0.0603 - accuracy: 0.9827 - val_loss: 0.4044 - val_accuracy: 0.9090
Epoch 6/15
180/180 - 20s - loss: 0.0529 - accuracy: 0.9842 - val_loss: 0.44

Epoch 2/15
180/180 - 19s - loss: 0.2021 - accuracy: 0.9343 - val_loss: 0.2348 - val_accuracy: 0.9040
Epoch 3/15
180/180 - 19s - loss: 0.0988 - accuracy: 0.9680 - val_loss: 0.3252 - val_accuracy: 0.9030
Epoch 4/15
180/180 - 19s - loss: 0.0709 - accuracy: 0.9799 - val_loss: 0.4325 - val_accuracy: 0.9020
Epoch 5/15
180/180 - 19s - loss: 0.0584 - accuracy: 0.9822 - val_loss: 0.5256 - val_accuracy: 0.8930
Epoch 6/15
180/180 - 19s - loss: 0.0511 - accuracy: 0.9820 - val_loss: 0.5402 - val_accuracy: 0.8960
Epoch 7/15
180/180 - 19s - loss: 0.0502 - accuracy: 0.9842 - val_loss: 0.4944 - val_accuracy: 0.9050
Epoch 8/15
180/180 - 19s - loss: 0.0522 - accuracy: 0.9827 - val_loss: 0.6147 - val_accuracy: 0.8920
Epoch 9/15
180/180 - 19s - loss: 0.0471 - accuracy: 0.9842 - val_loss: 0.7574 - val_accuracy: 0.8840
Epoch 10/15
180/180 - 19s - loss: 0.0503 - accuracy: 0.9818 - val_loss: 0.6477 - val_accuracy: 0.8990
Epoch 11/15
180/180 - 19s - loss: 0.0472 - accuracy: 0.9828 - val_loss: 0.6203 - val_accur

...

Test Accuracy: 92.1999990940094
Epoch 1/15
180/180 - 21s - loss: 0.4624 - accuracy: 0.7603 - val_loss: 0.2487 - val_accuracy: 0.9060
Epoch 2/15
180/180 - 20s - loss: 0.1517 - accuracy: 0.9576 - val_loss: 0.2494 - val_accuracy: 0.9070
Epoch 3/15
180/180 - 20s - loss: 0.0612 - accuracy: 0.9893 - val_loss: 0.2864 - val_accuracy: 0.9080
Epoch 4/15
180/180 - 20s - loss: 0.0322 - accuracy: 0.9970 - val_loss: 0.3203 - val_accuracy: 0.9110
Epoch 5/15
180/180 - 20s - loss: 0.0210 - accuracy: 0.9980 - val_loss: 0.3917 - val_accuracy: 0.8970
Epoch 6/15
180/180 - 20s - loss: 0.0152 - accuracy: 0.9994 - val_loss: 0.3946 - val_accuracy: 0.9060
Epoch 7/15
180/180 - 20s - loss: 0.0114 - accuracy: 0.9993 - val_loss: 0.4193 - val_accuracy: 0.9040
Epoch 8/15
180/180 - 20s - loss: 0.0099 - accuracy: 0.9991 - val_loss: 0.4608 - val_accuracy: 0.9060
Epoch 9/15
180/180 - 20s - loss: 0.0078 - accuracy: 0.9996 - val_loss: 0.4894 - val_accuracy: 0.9040
Restoring model weights from the end of the best epoch.
Epo

180/180 - 20s - loss: 0.0093 - accuracy: 0.9996 - val_loss: 0.4734 - val_accuracy: 0.8900
Epoch 7/15
180/180 - 21s - loss: 0.0074 - accuracy: 0.9996 - val_loss: 0.5084 - val_accuracy: 0.8900
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 90.20000100135803
Epoch 1/15
180/180 - 21s - loss: 0.4336 - accuracy: 0.7779 - val_loss: 0.2322 - val_accuracy: 0.9110
Epoch 2/15
180/180 - 21s - loss: 0.1306 - accuracy: 0.9628 - val_loss: 0.2167 - val_accuracy: 0.9130
Epoch 3/15
180/180 - 21s - loss: 0.0414 - accuracy: 0.9936 - val_loss: 0.2599 - val_accuracy: 0.9210
Epoch 4/15
180/180 - 20s - loss: 0.0198 - accuracy: 0.9978 - val_loss: 0.3441 - val_accuracy: 0.9050
Epoch 5/15
180/180 - 20s - loss: 0.0142 - accuracy: 0.9988 - val_loss: 0.3309 - val_accuracy: 0.9150
Epoch 6/15
180/180 - 20s - loss: 0.0105 - accuracy: 0.9991 - val_loss: 0.4047 - val_accuracy: 0.9080
Epoch 7/15
180/180 - 20s - loss: 0.0098 - accuracy: 0.9987 - val_loss: 0.4137 - val_ac

Epoch 7/15
180/180 - 22s - loss: 0.0061 - accuracy: 0.9996 - val_loss: 0.3492 - val_accuracy: 0.9180
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 93.30000281333923
Epoch 1/15
180/180 - 23s - loss: 0.4425 - accuracy: 0.7703 - val_loss: 0.2433 - val_accuracy: 0.8990
Epoch 2/15
180/180 - 22s - loss: 0.1391 - accuracy: 0.9584 - val_loss: 0.2033 - val_accuracy: 0.9230
Epoch 3/15
180/180 - 22s - loss: 0.0450 - accuracy: 0.9929 - val_loss: 0.2544 - val_accuracy: 0.9090
Epoch 4/15
180/180 - 22s - loss: 0.0196 - accuracy: 0.9983 - val_loss: 0.2836 - val_accuracy: 0.9170
Epoch 5/15
180/180 - 21s - loss: 0.0134 - accuracy: 0.9993 - val_loss: 0.3285 - val_accuracy: 0.9150
Epoch 6/15
180/180 - 22s - loss: 0.0109 - accuracy: 0.9992 - val_loss: 0.3459 - val_accuracy: 0.9170
Epoch 7/15
180/180 - 21s - loss: 0.0081 - accuracy: 0.9996 - val_loss: 0.3692 - val_accuracy: 0.9180
Restoring model weights from the end of the best epoch.
Epoch 00007: early 

Epoch 1/15
180/180 - 24s - loss: 0.4424 - accuracy: 0.7682 - val_loss: 0.2176 - val_accuracy: 0.9130
Epoch 2/15
180/180 - 23s - loss: 0.1263 - accuracy: 0.9649 - val_loss: 0.2321 - val_accuracy: 0.9190
Epoch 3/15
180/180 - 23s - loss: 0.0434 - accuracy: 0.9933 - val_loss: 0.2647 - val_accuracy: 0.9120
Epoch 4/15
180/180 - 22s - loss: 0.0176 - accuracy: 0.9990 - val_loss: 0.3189 - val_accuracy: 0.9150
Epoch 5/15
180/180 - 23s - loss: 0.0131 - accuracy: 0.9990 - val_loss: 0.3680 - val_accuracy: 0.9080
Epoch 6/15
180/180 - 23s - loss: 0.0092 - accuracy: 0.9994 - val_loss: 0.4061 - val_accuracy: 0.9110
Epoch 7/15
180/180 - 22s - loss: 0.0070 - accuracy: 0.9997 - val_loss: 0.4272 - val_accuracy: 0.9150
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 91.90000295639038
Epoch 1/15
180/180 - 24s - loss: 0.4358 - accuracy: 0.7762 - val_loss: 0.2455 - val_accuracy: 0.9040
Epoch 2/15
180/180 - 23s - loss: 0.1303 - accuracy: 0.9629 - val_loss: 0.25

Epoch 6/15
180/180 - 23s - loss: 0.0144 - accuracy: 0.9994 - val_loss: 0.3768 - val_accuracy: 0.9130
Epoch 7/15
180/180 - 23s - loss: 0.0107 - accuracy: 0.9993 - val_loss: 0.4014 - val_accuracy: 0.9100
Epoch 8/15
180/180 - 23s - loss: 0.0098 - accuracy: 0.9996 - val_loss: 0.4394 - val_accuracy: 0.9070
Epoch 9/15
180/180 - 23s - loss: 0.0124 - accuracy: 0.9970 - val_loss: 0.4309 - val_accuracy: 0.9050
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 92.00000166893005
Epoch 1/15
180/180 - 24s - loss: 0.4353 - accuracy: 0.7749 - val_loss: 0.2560 - val_accuracy: 0.8990
Epoch 2/15
180/180 - 23s - loss: 0.1411 - accuracy: 0.9602 - val_loss: 0.2175 - val_accuracy: 0.9210
Epoch 3/15
180/180 - 23s - loss: 0.0493 - accuracy: 0.9910 - val_loss: 0.2561 - val_accuracy: 0.9220
Epoch 4/15
180/180 - 23s - loss: 0.0268 - accuracy: 0.9972 - val_loss: 0.2660 - val_accuracy: 0.9250
Epoch 5/15
180/180 - 23s - loss: 0.0171 - accuracy: 0.9993 - val_loss: 0.30

## Summary

In [15]:
record.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
2,relu,3,91.600001,92.100000,91.500002,91.900003,90.499997,92.600000,92.299998,92.400002,92.500001,93.099999,92.050000
6,tanh,1,92.299998,92.400002,89.700001,91.900003,92.699999,91.299999,90.399998,92.600000,92.500001,93.800002,91.960000
0,relu,1,91.399997,92.600000,92.100000,91.700000,92.199999,92.299998,92.900002,91.700000,91.600001,90.300000,91.880000
1,relu,2,90.899998,92.000002,91.500002,92.400002,91.600001,91.600001,91.299999,92.699999,92.299998,92.000002,91.830000
10,tanh,5,91.799998,93.300003,92.299998,91.399997,90.899998,92.500001,91.799998,91.100001,90.799999,92.199999,91.809999
8,tanh,3,91.500002,92.000002,91.299999,93.000001,90.300000,91.900003,92.699999,91.600001,92.199999,91.100001,91.760001
5,relu,6,92.799997,91.600001,90.899998,91.900003,91.700000,93.099999,91.900003,91.100001,90.300000,91.600001,91.690000
7,tanh,2,91.700000,91.100001,90.899998,91.900003,91.299999,93.099999,92.299998,91.299999,91.200000,91.900003,91.670000
4,relu,5,91.100001,91.900003,90.799999,91.700000,92.699999,92.500001,91.700000,90.899998,91.200000,91.799998,91.630000
9,tanh,4,91.900003,90.700001,90.600002,92.500001,91.700000,90.200001,92.100000,90.799999,93.199998,92.100000,91.580001


In [16]:
record[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,92.05
tanh,91.96


In [17]:
report = record.sort_values(by='AVG', ascending=False)
report = report.to_excel('CNN_SUBJ.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [11]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [12]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [13]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [14]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 17913 words present from 21324 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [15]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()
print('emb_mean: ', emb_mean)
print('emb_std: ', emb_std)

emb_mean:  -0.003527845
emb_std:  0.13315111


In [16]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [17]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## CNN Model

In [18]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(filters = 100, kernel_size = 3, activation='relu', 
                 input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [19]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
conv1d (Conv1D)              (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 100)           0         
_________________________________________________________________
flatten (Flatten)            (None, 4900)              0         
_________________________________________________________________
dropout (Dropout)            (None, 4900)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                49010     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0

## Train and Test the Model

In [20]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [21]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

Epoch 1/100
180/180 [==============================] - 6s 31ms/step - loss: 0.6745 - accuracy: 0.5388 - val_loss: 0.3044 - val_accuracy: 0.8870
Epoch 2/100
180/180 [==============================] - 4s 25ms/step - loss: 0.3305 - accuracy: 0.8770 - val_loss: 0.2330 - val_accuracy: 0.9140
Epoch 3/100
180/180 [==============================] - 5s 28ms/step - loss: 0.2734 - accuracy: 0.8912 - val_loss: 0.2184 - val_accuracy: 0.9190
Epoch 4/100
180/180 [==============================] - 4s 24ms/step - loss: 0.2299 - accuracy: 0.9061 - val_loss: 0.2168 - val_accuracy: 0.9130
Epoch 5/100
180/180 [==============================] - 4s 24ms/step - loss: 0.2237 - accuracy: 0.9153 - val_loss: 0.2055 - val_accuracy: 0.9170
Epoch 6/100
180/180 [==============================] - 4s 23ms/step - loss: 0.1939 - accuracy: 0.9251 - val_loss: 0.2104 - val_accuracy: 0.9120
Epoch 7/100
180/180 [==============================] - 4s 23ms/step - loss: 0.1931 - accuracy: 0.9279 - val_loss: 0.2082 - val_accuracy:

180/180 [==============================] - 5s 25ms/step - loss: 0.2577 - accuracy: 0.9030 - val_loss: 0.2350 - val_accuracy: 0.9030
Epoch 5/100
180/180 [==============================] - 5s 27ms/step - loss: 0.2473 - accuracy: 0.9108 - val_loss: 0.2357 - val_accuracy: 0.9070
Epoch 6/100
180/180 [==============================] - 4s 24ms/step - loss: 0.2156 - accuracy: 0.9209 - val_loss: 0.2369 - val_accuracy: 0.9070
Epoch 7/100
180/180 [==============================] - 4s 23ms/step - loss: 0.2029 - accuracy: 0.9172 - val_loss: 0.2300 - val_accuracy: 0.9060
Epoch 8/100
180/180 [==============================] - 4s 24ms/step - loss: 0.1837 - accuracy: 0.9289 - val_loss: 0.2335 - val_accuracy: 0.9070
Epoch 9/100
180/180 [==============================] - 4s 22ms/step - loss: 0.1694 - accuracy: 0.9351 - val_loss: 0.2311 - val_accuracy: 0.9090
Epoch 10/100
180/180 [==============================] - 4s 23ms/step - loss: 0.1661 - accuracy: 0.9323 - val_loss: 0.2376 - val_accuracy: 0.9070
Epo

180/180 [==============================] - 5s 26ms/step - loss: 0.1447 - accuracy: 0.9532 - val_loss: 0.2518 - val_accuracy: 0.9230
Epoch 14/100
180/180 [==============================] - 4s 22ms/step - loss: 0.1299 - accuracy: 0.9588 - val_loss: 0.2520 - val_accuracy: 0.9210
Epoch 15/100
180/180 [==============================] - 4s 22ms/step - loss: 0.1230 - accuracy: 0.9587 - val_loss: 0.2625 - val_accuracy: 0.9220
Epoch 16/100
180/180 [==============================] - 4s 23ms/step - loss: 0.1197 - accuracy: 0.9621 - val_loss: 0.2657 - val_accuracy: 0.9190
Epoch 17/100
180/180 [==============================] - 4s 22ms/step - loss: 0.1055 - accuracy: 0.9670 - val_loss: 0.2514 - val_accuracy: 0.9160
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Test Accuracy: 93.00000071525574
Epoch 1/100
180/180 [==============================] - 5s 23ms/step - loss: 0.6858 - accuracy: 0.5218 - val_loss: 0.3856 - val_accuracy: 0.8710
Epoch 2/100
180/180 [======

180/180 [==============================] - 4s 23ms/step - loss: 0.1066 - accuracy: 0.9582 - val_loss: 0.2322 - val_accuracy: 0.9200
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
Test Accuracy: 93.00000071525574
Epoch 1/100
180/180 [==============================] - 4s 18ms/step - loss: 0.6570 - accuracy: 0.5701 - val_loss: 0.3430 - val_accuracy: 0.8960
Epoch 2/100
180/180 [==============================] - 3s 17ms/step - loss: 0.3905 - accuracy: 0.8068 - val_loss: 0.2459 - val_accuracy: 0.9030
Epoch 3/100
180/180 [==============================] - 3s 17ms/step - loss: 0.3170 - accuracy: 0.8884 - val_loss: 0.2265 - val_accuracy: 0.9090
Epoch 4/100
180/180 [==============================] - 3s 17ms/step - loss: 0.2545 - accuracy: 0.9082 - val_loss: 0.2083 - val_accuracy: 0.9150
Epoch 5/100
180/180 [==============================] - 3s 18ms/step - loss: 0.2376 - accuracy: 0.9239 - val_loss: 0.1981 - val_accuracy: 0.9200
Epoch 6/100
180/180 [==========

180/180 [==============================] - 4s 22ms/step - loss: 0.2398 - accuracy: 0.9097 - val_loss: 0.1895 - val_accuracy: 0.9280
Epoch 8/100
180/180 [==============================] - 5s 27ms/step - loss: 0.2113 - accuracy: 0.9241 - val_loss: 0.1844 - val_accuracy: 0.9230
Epoch 9/100
180/180 [==============================] - 4s 25ms/step - loss: 0.2005 - accuracy: 0.9267 - val_loss: 0.1868 - val_accuracy: 0.9320
Epoch 10/100
180/180 [==============================] - 4s 22ms/step - loss: 0.1730 - accuracy: 0.9397 - val_loss: 0.1856 - val_accuracy: 0.9220
Epoch 11/100
180/180 [==============================] - 4s 24ms/step - loss: 0.1751 - accuracy: 0.9365 - val_loss: 0.1960 - val_accuracy: 0.9220
Epoch 12/100
180/180 [==============================] - 4s 22ms/step - loss: 0.1518 - accuracy: 0.9451 - val_loss: 0.1805 - val_accuracy: 0.9280
Epoch 13/100
180/180 [==============================] - 4s 22ms/step - loss: 0.1495 - accuracy: 0.9437 - val_loss: 0.1959 - val_accuracy: 0.9290


...

180/180 [==============================] - 11s 61ms/step - loss: 0.1538 - accuracy: 0.9513 - val_loss: 0.2825 - val_accuracy: 0.9030
Epoch 8/100
180/180 [==============================] - 11s 62ms/step - loss: 0.1306 - accuracy: 0.9634 - val_loss: 0.3113 - val_accuracy: 0.9000
Epoch 9/100
180/180 [==============================] - 11s 62ms/step - loss: 0.1237 - accuracy: 0.9642 - val_loss: 0.3077 - val_accuracy: 0.9030
Epoch 10/100
180/180 [==============================] - 11s 61ms/step - loss: 0.1111 - accuracy: 0.9682 - val_loss: 0.3385 - val_accuracy: 0.8980
Epoch 11/100
180/180 [==============================] - 12s 65ms/step - loss: 0.0985 - accuracy: 0.9728 - val_loss: 0.3649 - val_accuracy: 0.9020
Epoch 12/100
180/180 [==============================] - 11s 62ms/step - loss: 0.0967 - accuracy: 0.9729 - val_loss: 0.3743 - val_accuracy: 0.9020
Epoch 13/100
180/180 [==============================] - 11s 62ms/step - loss: 0.0834 - accuracy: 0.9778 - val_loss: 0.3949 - val_accuracy: 

180/180 [==============================] - 15s 81ms/step - loss: 0.6864 - accuracy: 0.5254 - val_loss: 0.3459 - val_accuracy: 0.8630
Epoch 2/100
180/180 [==============================] - 22s 122ms/step - loss: 0.3774 - accuracy: 0.8493 - val_loss: 0.2465 - val_accuracy: 0.9030
Epoch 3/100
180/180 [==============================] - 22s 122ms/step - loss: 0.2806 - accuracy: 0.8885 - val_loss: 0.2242 - val_accuracy: 0.9120
Epoch 4/100
180/180 [==============================] - 22s 120ms/step - loss: 0.2276 - accuracy: 0.9016 - val_loss: 0.2562 - val_accuracy: 0.8990
Epoch 5/100
180/180 [==============================] - 21s 119ms/step - loss: 0.1838 - accuracy: 0.9200 - val_loss: 0.2400 - val_accuracy: 0.9080
Epoch 6/100
180/180 [==============================] - 21s 118ms/step - loss: 0.1408 - accuracy: 0.9332 - val_loss: 0.2316 - val_accuracy: 0.9110
Epoch 7/100
180/180 [==============================] - 22s 120ms/step - loss: 0.1218 - accuracy: 0.9408 - val_loss: 0.2525 - val_accuracy

180/180 [==============================] - 16s 91ms/step - loss: 0.1084 - accuracy: 0.9673 - val_loss: 0.3016 - val_accuracy: 0.9050
Epoch 9/100
180/180 [==============================] - 16s 91ms/step - loss: 0.0965 - accuracy: 0.9736 - val_loss: 0.3298 - val_accuracy: 0.9070
Epoch 10/100
180/180 [==============================] - 17s 93ms/step - loss: 0.0853 - accuracy: 0.9758 - val_loss: 0.3374 - val_accuracy: 0.9080
Epoch 11/100
180/180 [==============================] - 17s 93ms/step - loss: 0.0704 - accuracy: 0.9802 - val_loss: 0.3491 - val_accuracy: 0.9100
Epoch 12/100
180/180 [==============================] - 17s 94ms/step - loss: 0.0719 - accuracy: 0.9786 - val_loss: 0.4372 - val_accuracy: 0.9020
Epoch 13/100
180/180 [==============================] - 17s 92ms/step - loss: 0.0639 - accuracy: 0.9843 - val_loss: 0.4127 - val_accuracy: 0.9030
Epoch 14/100
180/180 [==============================] - 16s 91ms/step - loss: 0.0605 - accuracy: 0.9829 - val_loss: 0.4229 - val_accuracy:

180/180 [==============================] - 12s 69ms/step - loss: 0.0799 - accuracy: 0.9858 - val_loss: 0.3445 - val_accuracy: 0.9140
Epoch 12/100
180/180 [==============================] - 13s 70ms/step - loss: 0.0830 - accuracy: 0.9831 - val_loss: 0.3066 - val_accuracy: 0.9190
Epoch 13/100
180/180 [==============================] - 13s 70ms/step - loss: 0.0635 - accuracy: 0.9889 - val_loss: 0.3470 - val_accuracy: 0.9190
Epoch 14/100
180/180 [==============================] - 12s 68ms/step - loss: 0.0624 - accuracy: 0.9869 - val_loss: 0.3303 - val_accuracy: 0.9210
Epoch 15/100
180/180 [==============================] - 13s 72ms/step - loss: 0.0570 - accuracy: 0.9871 - val_loss: 0.3095 - val_accuracy: 0.9140
Epoch 16/100
180/180 [==============================] - 13s 73ms/step - loss: 0.0561 - accuracy: 0.9896 - val_loss: 0.3586 - val_accuracy: 0.9170
Epoch 17/100
180/180 [==============================] - 12s 69ms/step - loss: 0.0573 - accuracy: 0.9871 - val_loss: 0.3573 - val_accuracy

Epoch 16/100
180/180 [==============================] - 12s 67ms/step - loss: 0.0558 - accuracy: 0.9820 - val_loss: 0.4264 - val_accuracy: 0.9120
Epoch 17/100
180/180 [==============================] - 12s 68ms/step - loss: 0.0508 - accuracy: 0.9816 - val_loss: 0.4411 - val_accuracy: 0.9010
Epoch 18/100
180/180 [==============================] - 12s 68ms/step - loss: 0.0500 - accuracy: 0.9835 - val_loss: 0.4095 - val_accuracy: 0.9030
Epoch 19/100
180/180 [==============================] - 12s 67ms/step - loss: 0.0431 - accuracy: 0.9869 - val_loss: 0.4044 - val_accuracy: 0.8980
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Test Accuracy: 91.29999876022339
Epoch 1/100
180/180 [==============================] - 13s 70ms/step - loss: 0.6982 - accuracy: 0.5046 - val_loss: 0.6933 - val_accuracy: 0.4870
Epoch 2/100
180/180 [==============================] - 12s 69ms/step - loss: 0.6931 - accuracy: 0.5006 - val_loss: 0.6932 - val_accuracy: 0.4870
Epoch 3/1

## Summary

In [22]:
record2.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,relu,1,92.600000,93.599999,91.500002,92.299998,93.000001,91.500002,93.000001,93.000001,91.399997,93.199998,92.510000
1,relu,2,92.799997,92.699999,91.700000,92.699999,93.500000,92.199999,91.500002,93.300003,91.399997,91.900003,92.370000
2,relu,3,91.000003,92.299998,94.099998,92.500001,92.400002,91.799998,91.100001,91.100001,92.400002,92.100000,92.080001
3,relu,4,91.500002,92.199999,91.299999,91.600001,92.299998,90.799999,92.900002,91.399997,91.700000,91.500002,91.720000
4,relu,5,92.699999,92.199999,91.000003,92.000002,91.399997,93.400002,91.399997,92.000002,50.400001,90.799999,87.730000
5,relu,6,91.700000,92.799997,90.600002,92.799997,90.100002,51.400000,92.100000,90.300000,90.399998,91.600001,87.380000
6,relu,7,90.899998,91.399997,91.200000,48.600000,91.500002,91.799998,92.699999,90.600002,91.100001,91.100001,87.090000
7,relu,8,91.200000,92.199999,91.000003,92.100000,92.100000,91.299999,45.899999,91.299999,90.899998,91.600001,86.960000


In [23]:
record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,92.51


In [24]:
report = record2.sort_values(by='AVG', ascending=False)
report = report.to_excel('CNN_SUBJ_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## CNN Model

In [25]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(filters = 100, kernel_size = 3, activation='relu', 
                 input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [26]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_81"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_81 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
conv1d_81 (Conv1D)           (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d_81 (MaxPooling (None, 49, 100)           0         
_________________________________________________________________
flatten_81 (Flatten)         (None, 4900)              0         
_________________________________________________________________
dropout_162 (Dropout)        (None, 4900)              0         
_________________________________________________________________
dense_162 (Dense)            (None, 10)                49010     
_________________________________________________________________
dropout_163 (Dropout)        (None, 10)              

## Train and Test the Model

In [27]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [29]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

Epoch 1/100
180/180 [==============================] - 21s 108ms/step - loss: 0.6861 - accuracy: 0.5140 - val_loss: 0.4301 - val_accuracy: 0.8850
Epoch 2/100
180/180 [==============================] - 19s 108ms/step - loss: 0.4127 - accuracy: 0.7990 - val_loss: 0.2216 - val_accuracy: 0.9080
Epoch 3/100
180/180 [==============================] - 18s 99ms/step - loss: 0.1936 - accuracy: 0.8990 - val_loss: 0.2163 - val_accuracy: 0.9080
Epoch 4/100
180/180 [==============================] - 21s 116ms/step - loss: 0.1200 - accuracy: 0.9184 - val_loss: 0.2668 - val_accuracy: 0.8980
Epoch 5/100
180/180 [==============================] - 19s 108ms/step - loss: 0.0905 - accuracy: 0.9378 - val_loss: 0.3304 - val_accuracy: 0.9040
Epoch 6/100
180/180 [==============================] - 20s 109ms/step - loss: 0.0619 - accuracy: 0.9603 - val_loss: 0.4085 - val_accuracy: 0.9030
Epoch 7/100
180/180 [==============================] - 20s 112ms/step - loss: 0.0501 - accuracy: 0.9710 - val_loss: 0.4206 - 

180/180 [==============================] - 19s 104ms/step - loss: 0.0318 - accuracy: 0.9849 - val_loss: 0.4236 - val_accuracy: 0.9210
Epoch 15/100
180/180 [==============================] - 19s 104ms/step - loss: 0.0276 - accuracy: 0.9876 - val_loss: 0.5033 - val_accuracy: 0.9190
Epoch 16/100
180/180 [==============================] - 19s 103ms/step - loss: 0.0235 - accuracy: 0.9917 - val_loss: 0.5527 - val_accuracy: 0.9150
Epoch 17/100
180/180 [==============================] - 19s 103ms/step - loss: 0.0229 - accuracy: 0.9932 - val_loss: 0.6149 - val_accuracy: 0.9170
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Test Accuracy: 93.59999895095825
Epoch 1/100
180/180 [==============================] - 21s 115ms/step - loss: 0.6571 - accuracy: 0.5609 - val_loss: 0.2431 - val_accuracy: 0.9050
Epoch 2/100
180/180 [==============================] - 18s 101ms/step - loss: 0.2626 - accuracy: 0.8963 - val_loss: 0.1979 - val_accuracy: 0.9230
Epoch 3/100
180/

Epoch 12/100
180/180 [==============================] - 18s 103ms/step - loss: 0.0510 - accuracy: 0.9832 - val_loss: 0.4455 - val_accuracy: 0.9190
Epoch 13/100
180/180 [==============================] - 18s 102ms/step - loss: 0.0467 - accuracy: 0.9849 - val_loss: 0.5092 - val_accuracy: 0.9200
Epoch 14/100
180/180 [==============================] - 18s 102ms/step - loss: 0.0458 - accuracy: 0.9825 - val_loss: 0.5917 - val_accuracy: 0.9170
Epoch 15/100
180/180 [==============================] - 18s 100ms/step - loss: 0.0434 - accuracy: 0.9831 - val_loss: 0.6010 - val_accuracy: 0.9080
Epoch 16/100
180/180 [==============================] - 18s 100ms/step - loss: 0.0441 - accuracy: 0.9847 - val_loss: 0.6201 - val_accuracy: 0.9100
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Test Accuracy: 93.30000281333923
Epoch 1/100
180/180 [==============================] - 19s 103ms/step - loss: 0.6130 - accuracy: 0.6350 - val_loss: 0.2465 - val_accuracy: 0.9050
Ep

Epoch 10/100
180/180 [==============================] - 21s 118ms/step - loss: 0.0288 - accuracy: 0.9969 - val_loss: 0.5287 - val_accuracy: 0.9230
Epoch 11/100
180/180 [==============================] - 21s 117ms/step - loss: 0.0284 - accuracy: 0.9942 - val_loss: 0.6654 - val_accuracy: 0.9250
Epoch 12/100
180/180 [==============================] - 20s 109ms/step - loss: 0.0338 - accuracy: 0.9926 - val_loss: 0.5439 - val_accuracy: 0.9260
Epoch 13/100
180/180 [==============================] - 20s 111ms/step - loss: 0.0263 - accuracy: 0.9945 - val_loss: 0.6438 - val_accuracy: 0.9210
Epoch 14/100
180/180 [==============================] - 20s 110ms/step - loss: 0.0228 - accuracy: 0.9969 - val_loss: 0.6016 - val_accuracy: 0.9280
Epoch 15/100
180/180 [==============================] - 20s 111ms/step - loss: 0.0320 - accuracy: 0.9923 - val_loss: 0.7611 - val_accuracy: 0.9270
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
Test Accuracy: 93.30000281333923
E

Epoch 8/100
180/180 [==============================] - 20s 110ms/step - loss: 0.0461 - accuracy: 0.9882 - val_loss: 0.4806 - val_accuracy: 0.9040
Epoch 9/100
180/180 [==============================] - 20s 111ms/step - loss: 0.0383 - accuracy: 0.9924 - val_loss: 0.5544 - val_accuracy: 0.9110
Epoch 10/100
180/180 [==============================] - 20s 111ms/step - loss: 0.0377 - accuracy: 0.9935 - val_loss: 0.5153 - val_accuracy: 0.9110
Epoch 11/100
180/180 [==============================] - 20s 110ms/step - loss: 0.0410 - accuracy: 0.9895 - val_loss: 0.5099 - val_accuracy: 0.9090
Epoch 12/100
180/180 [==============================] - 20s 111ms/step - loss: 0.0349 - accuracy: 0.9905 - val_loss: 0.6287 - val_accuracy: 0.9170
Epoch 13/100
180/180 [==============================] - 20s 112ms/step - loss: 0.0326 - accuracy: 0.9915 - val_loss: 0.6340 - val_accuracy: 0.9080
Epoch 14/100
180/180 [==============================] - 20s 112ms/step - loss: 0.0335 - accuracy: 0.9912 - val_loss: 0.6

...

180/180 [==============================] - 25s 136ms/step - loss: 0.0527 - accuracy: 0.9966 - val_loss: 0.5875 - val_accuracy: 0.9220
Epoch 14/100
180/180 [==============================] - 23s 130ms/step - loss: 0.0429 - accuracy: 0.9983 - val_loss: 0.6364 - val_accuracy: 0.9190
Epoch 15/100
180/180 [==============================] - 23s 130ms/step - loss: 0.0431 - accuracy: 0.9965 - val_loss: 0.8431 - val_accuracy: 0.9090
Epoch 16/100
180/180 [==============================] - 23s 130ms/step - loss: 0.0392 - accuracy: 0.9980 - val_loss: 0.4855 - val_accuracy: 0.9210
Epoch 17/100
180/180 [==============================] - 23s 129ms/step - loss: 0.0388 - accuracy: 0.9969 - val_loss: 0.9178 - val_accuracy: 0.9170
Epoch 18/100
180/180 [==============================] - 23s 129ms/step - loss: 0.0335 - accuracy: 0.9974 - val_loss: 0.7709 - val_accuracy: 0.9190
Epoch 19/100
180/180 [==============================] - 25s 142ms/step - loss: 0.0304 - accuracy: 0.9980 - val_loss: 0.7480 - val_a

180/180 [==============================] - 25s 138ms/step - loss: 0.0318 - accuracy: 0.9813 - val_loss: 0.5443 - val_accuracy: 0.9050
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
Test Accuracy: 92.29999780654907
Epoch 1/100
180/180 [==============================] - 25s 133ms/step - loss: 0.5993 - accuracy: 0.6279 - val_loss: 0.2129 - val_accuracy: 0.9170
Epoch 2/100
180/180 [==============================] - 21s 119ms/step - loss: 0.1897 - accuracy: 0.9256 - val_loss: 0.2177 - val_accuracy: 0.9120
Epoch 3/100
180/180 [==============================] - 22s 120ms/step - loss: 0.0859 - accuracy: 0.9679 - val_loss: 0.2797 - val_accuracy: 0.9210
Epoch 4/100
180/180 [==============================] - 21s 119ms/step - loss: 0.0477 - accuracy: 0.9770 - val_loss: 0.3676 - val_accuracy: 0.9150
Epoch 5/100
180/180 [==============================] - 22s 120ms/step - loss: 0.0341 - accuracy: 0.9821 - val_loss: 0.4067 - val_accuracy: 0.9170
Epoch 6/100
180/180

Epoch 2/100
180/180 [==============================] - 23s 130ms/step - loss: 0.4262 - accuracy: 0.7934 - val_loss: 0.2165 - val_accuracy: 0.9200
Epoch 3/100
180/180 [==============================] - 23s 129ms/step - loss: 0.1423 - accuracy: 0.9384 - val_loss: 0.1970 - val_accuracy: 0.9310
Epoch 4/100
180/180 [==============================] - 23s 130ms/step - loss: 0.0735 - accuracy: 0.9579 - val_loss: 0.2508 - val_accuracy: 0.9320
Epoch 5/100
180/180 [==============================] - 24s 131ms/step - loss: 0.0532 - accuracy: 0.9783 - val_loss: 0.3230 - val_accuracy: 0.9200
Epoch 6/100
180/180 [==============================] - 23s 130ms/step - loss: 0.0406 - accuracy: 0.9820 - val_loss: 0.3715 - val_accuracy: 0.9270
Epoch 7/100
180/180 [==============================] - 23s 130ms/step - loss: 0.0395 - accuracy: 0.9821 - val_loss: 0.4011 - val_accuracy: 0.9250
Epoch 8/100
180/180 [==============================] - 23s 130ms/step - loss: 0.0318 - accuracy: 0.9911 - val_loss: 0.4827 -

Epoch 6/100
180/180 [==============================] - 22s 122ms/step - loss: 0.0480 - accuracy: 0.9916 - val_loss: 0.3759 - val_accuracy: 0.9270
Epoch 7/100
180/180 [==============================] - 24s 133ms/step - loss: 0.0422 - accuracy: 0.9897 - val_loss: 0.4156 - val_accuracy: 0.9170
Epoch 8/100
180/180 [==============================] - 22s 122ms/step - loss: 0.0417 - accuracy: 0.9894 - val_loss: 0.3974 - val_accuracy: 0.9250
Epoch 9/100
180/180 [==============================] - 22s 123ms/step - loss: 0.0368 - accuracy: 0.9909 - val_loss: 0.4636 - val_accuracy: 0.9250
Epoch 10/100
180/180 [==============================] - 22s 122ms/step - loss: 0.0364 - accuracy: 0.9914 - val_loss: 0.5174 - val_accuracy: 0.9230
Epoch 11/100
180/180 [==============================] - 22s 122ms/step - loss: 0.0327 - accuracy: 0.9937 - val_loss: 0.5022 - val_accuracy: 0.9260
Epoch 12/100
180/180 [==============================] - 22s 123ms/step - loss: 0.0299 - accuracy: 0.9933 - val_loss: 0.595

180/180 [==============================] - 36s 203ms/step - loss: 0.0302 - accuracy: 0.9848 - val_loss: 0.5184 - val_accuracy: 0.9240
Epoch 18/100
180/180 [==============================] - 36s 198ms/step - loss: 0.0307 - accuracy: 0.9868 - val_loss: 0.5115 - val_accuracy: 0.9240
Epoch 19/100
180/180 [==============================] - 36s 198ms/step - loss: 0.0320 - accuracy: 0.9848 - val_loss: 0.5673 - val_accuracy: 0.9210
Epoch 20/100
180/180 [==============================] - 36s 198ms/step - loss: 0.0360 - accuracy: 0.9806 - val_loss: 0.6058 - val_accuracy: 0.9250
Epoch 21/100
180/180 [==============================] - 36s 198ms/step - loss: 0.0316 - accuracy: 0.9835 - val_loss: 0.6872 - val_accuracy: 0.9260
Epoch 22/100
180/180 [==============================] - 36s 197ms/step - loss: 0.0339 - accuracy: 0.9815 - val_loss: 0.6699 - val_accuracy: 0.9240
Epoch 23/100
180/180 [==============================] - 36s 197ms/step - loss: 0.0314 - accuracy: 0.9874 - val_loss: 0.6497 - val_a

## Summary

In [30]:
record3.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,relu,1,91.299999,93.900001,93.599999,93.099999,91.500002,93.400002,93.300003,92.799997,92.699999,92.699999,92.830000
1,relu,2,93.300003,92.799997,92.900002,94.300002,91.700000,93.099999,92.600000,91.600001,92.600000,91.600001,92.650000
4,relu,5,92.199999,91.399997,93.300003,91.900003,92.900002,93.000001,92.199999,93.000001,92.699999,91.600001,92.420000
2,relu,3,94.099998,50.500000,92.400002,92.699999,92.699999,91.700000,92.600000,91.200000,91.900003,93.500000,88.330000
7,relu,8,92.799997,93.199998,91.100001,50.300002,91.399997,93.300003,90.799999,92.199999,93.500000,93.199998,88.180000
3,relu,4,48.400000,91.299999,92.900002,93.000001,92.900002,91.399997,92.500001,92.299998,92.000002,92.600000,87.930000
5,relu,6,91.500002,50.500000,92.400002,91.900003,91.900003,50.900000,92.400002,92.799997,93.300003,92.199999,83.980001
6,relu,7,49.399999,48.800001,50.400001,92.299998,93.000001,93.400002,47.799999,92.299998,92.100000,91.299999,75.080000


In [31]:
report = record3.sort_values(by='AVG', ascending=False)
report = report.to_excel('CNN_SUBJ_3.xlsx', sheet_name='dynamic')